In [72]:
# importing packages and libraries
from mxnet import nd
from mxnet.gluon import nn
from mxnet import autograd
from mxnet import gluon
from mxnet import np, npx
import pandas as pd
import mxnet as mx
# npx.set_np()
data_ctx = mx.cpu()
model_ctx = mx.cpu()

In [73]:
# Load the IRIS dataset using pandas
df = pd.read_csv('iris.csv')

# Extract features and labels
X = df.iloc[:, :-1].values.astype(np.float32)
y_labels = df.iloc[:, -1].values

# encode the string class values as integers
y_labels[y_labels == 'Iris-setosa'] = 0.0
y_labels[y_labels == 'Iris-versicolor'] = 1.0
y_labels[y_labels == 'Iris-virginica'] = 2.0

# convert to mxnet.numpy.ndarray
X = nd.array(X)
y = nd.array(y_labels)

In [74]:
# Split the dataset into training and testing sets
X_train = X[:120]
X_test = X[120:]
y_train = y[:120]
y_test = y[120:]

In [75]:
# Define the data iterator
batch_size = 10
train_data = gluon.data.DataLoader(gluon.data.ArrayDataset(X_train, y_train), batch_size=batch_size, shuffle=True)

In [76]:
# Define the model
num_inputs = X_train.shape[1]
num_outputs = 3
net = gluon.nn.Sequential()
net.add(gluon.nn.Dense(num_outputs))

In [77]:
# Initialize the weights
net.initialize(mx.init.Normal(sigma=0.01))

In [78]:
# Define the loss function
loss_function = gluon.loss.SoftmaxCrossEntropyLoss()

In [79]:
# Define the optimizer
optimizer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.01})

In [82]:
# Train the model
epochs = 100
for epoch in range(epochs):
    cumulative_loss = 0  # loss for every epoch
    for X_batch, y_batch in train_data:
        with autograd.record():     # record the gradients
            output = net(X_batch)   # forward pass
            loss = loss_function(output, y_batch)   # calculate the loss
        loss.backward()        # backward pass
        optimizer.step(batch_size)   # updating the weights and bias
        cumulative_loss += nd.mean(loss).asscalar()    # calculate the cumulative loss
    accuracy = (net(X_train).argmax(axis=1) == y_train).mean().asscalar()
    softmax_loss = nd.mean(loss).asscalar()
    print(f"Epoch: {epoch+1}, Softmax Loss: {softmax_loss:.4f}, Accuracy: {accuracy * 100}%")

Epoch: 1, Softmax Loss: 15.5255, Accuracy: 75.83333253860474%
Epoch: 2, Softmax Loss: 1.3215, Accuracy: 80.83333373069763%
Epoch: 3, Softmax Loss: 15.8061, Accuracy: 70.83333134651184%
Epoch: 4, Softmax Loss: 4.5376, Accuracy: 58.33333134651184%
Epoch: 5, Softmax Loss: 10.6335, Accuracy: 80.0000011920929%
Epoch: 6, Softmax Loss: 1.8914, Accuracy: 71.66666388511658%
Epoch: 7, Softmax Loss: 0.3778, Accuracy: 75.0%
Epoch: 8, Softmax Loss: 41.0203, Accuracy: 74.16666746139526%
Epoch: 9, Softmax Loss: 10.9027, Accuracy: 86.66666746139526%
Epoch: 10, Softmax Loss: 13.0635, Accuracy: 49.166667461395264%
Epoch: 11, Softmax Loss: 0.8404, Accuracy: 75.0%
Epoch: 12, Softmax Loss: 1.3568, Accuracy: 58.33333134651184%
Epoch: 13, Softmax Loss: 1.6326, Accuracy: 73.33333492279053%
Epoch: 14, Softmax Loss: 0.0124, Accuracy: 83.33333134651184%
Epoch: 15, Softmax Loss: 0.0028, Accuracy: 83.33333134651184%
Epoch: 16, Softmax Loss: 6.4850, Accuracy: 83.33333134651184%
Epoch: 17, Softmax Loss: 7.3223, Accu

In [85]:
# Evaluate the model
# testing the model against the test dataset
test_output = net(X_test)
# predicted labels are the maximum value of the output along the axis 1
predicted_labels = nd.argmax(test_output, axis=1)
# calculate the accuracy of the model
accuracy = (predicted_labels == y_test).mean().asscalar()
print(f"Test Accuracy: {accuracy * 100}%")

Test Accuracy: 100.0%
